In [1]:
from glob import glob
import os
import pandas as pd
from subprocess import Popen, PIPE, run

import requests

In [2]:
ls

ABIDE_Initiative-clean.csv           fcp-indi-new
ABIDE_Initiative.csv                 fcp-indi-new.gz
ACPI-clean.csv                       fcp-info.ipynb
ACPI.csv                             HypnosisBarrios-clean.csv
ADHD200-clean.csv                    HypnosisBarrios.csv
ADHD200.csv                          load/
aseg.mgz                             pheno/
BrainGenomicsSuperstructProject.csv  process-clean-csv.ipynb
CORR-clean.csv                       process-csv.ipynb
CORR.csv                             RocklandSample-clean.csv
extract/                             RocklandSample.csv
fcp-indi                             transform/


In [8]:
csv_files = [val for val in sorted(glob('*.csv')) if not any([key in val for key in 
                                                              ['clean', 'Brain', 'ABIDE']])]
csv_files

['ACPI.csv',
 'ADHD200.csv',
 'CORR.csv',
 'HypnosisBarrios.csv',
 'RocklandSample.csv']

In [4]:
def process_csv(filename):
    #filename = csv_files[0]
    df = pd.read_csv(filename, header=None, names=['url', 'id'], dtype=str)
    print(filename, df.shape)
    info = []
    for grp in df.groupby('id'):
        T1 = [val for val in grp[1].url.values if 'T1.mgz' in val]
        if T1:
            for T1_val in T1:
                annot = T1_val.replace('T1.mgz', 'aseg.mgz')
                this_info = dict(url=T1_val, annot=annot, id=grp[0])
                info.append(this_info)
        else:
            for val in grp[1].url.values:
                this_info = dict(url=val, annot=pd.np.nan, id=grp[0])
                info.append(this_info)
    new_df = pd.DataFrame(info)
    return new_df

In [9]:
all_df = []
for filename in csv_files:
    new_df = process_csv(filename)
    new_filename = filename.replace('.csv', '-clean.csv')
    new_df[['url', 'id']].to_csv(new_filename, header=False, index=False)
    print(new_filename, new_df.shape)
    all_df.append(new_df)
all_df = pd.concat(all_df)
all_df.shape

ACPI.csv (129, 2)
ACPI-clean.csv (129, 3)
ADHD200.csv (1927, 2)
ADHD200-clean.csv (973, 3)
CORR.csv (3203, 2)
CORR-clean.csv (3153, 3)
HypnosisBarrios.csv (10, 2)
HypnosisBarrios-clean.csv (10, 3)
RocklandSample.csv (532, 2)
RocklandSample-clean.csv (532, 3)


(4797, 3)